<a href="https://colab.research.google.com/github/Sowmithaa22/BMS/blob/main/Cyber_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PHISHING WEBSITE DETECTION**

# Importing the necessary libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.e nsemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score


# 📌 Step 1: Load the Dataset

In [ ]:

df = pd.read_csv("/content/PhiUSIIL_Phishing_URL_Dataset.csv")  # Update with actual dataset path


# 📌 Step 2: Data Exploration

In [ ]:
print(df.info())  # Check data types and missing values
print(df.describe())  # Summary statistics

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235795 entries, 0 to 235794
Data columns (total 56 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   FILENAME                    235795 non-null  object 
 1   URL                         235795 non-null  object 
 2   URLLength                   235795 non-null  int64  
 3   Domain                      235795 non-null  object 
 4   DomainLength                235795 non-null  int64  
 5   IsDomainIP                  235795 non-null  int64  
 6   TLD                         235795 non-null  object 
 7   URLSimilarityIndex          235795 non-null  float64
 8   CharContinuationRate        235795 non-null  float64
 9   TLDLegitimateProb           235795 non-null  float64
 10  URLCharProb                 235795 non-null  float64
 11  TLDLength                   235795 non-null  int64  
 12  NoOfSubDomain               235795 non-null  int64  
 13  HasObfuscation

# 📌 Step 3: Handle Missing Values (if any)

In [ ]:
df.isnull().sum  # Replace NaN with median values

<bound method DataFrame.sum of         FILENAME    URL  URLLength  Domain  DomainLength  IsDomainIP    TLD  \
0          False  False      False   False         False       False  False   
1          False  False      False   False         False       False  False   
2          False  False      False   False         False       False  False   
3          False  False      False   False         False       False  False   
4          False  False      False   False         False       False  False   
...          ...    ...        ...     ...           ...         ...    ...   
235790     False  False      False   False         False       False  False   
235791     False  False      False   False         False       False  False   
235792     False  False      False   False         False       False  False   
235793     False  False      False   False         False       False  False   
235794     False  False      False   False         False       False  False   

        URLSimilarityIndex  CharContinuationRate  TLDLegitimateProb  ...  \
0                    False                 False              False  ...   
1                    False                 False              False  ...   
2                    False                 False              False  ...   
3                    False                 False              False  ...   
4                    False                 False              False  ...   
...                    ...                   ...                ...  ...   
235790               False                 False              False  ...   
235791               False                 False              False  ...   
235792               False                 False              False  ...   
235793               False                 False              False  ...   
235794               False                 False              False  ...   

          Pay  Crypto  HasCopyrightInfo  NoOfImage  NoOfCSS  NoOfJS  \
0       False   False             False      False    False   False   
1       False   False             False      False    False   False   
2       False   False             False      False    False   False   
3       False   False             False      False    False   False   
4       False   False             False      False    False   False   
...       ...     ...               ...        ...      ...     ...   
235790  False   False             False      False    False   False   
235791  False   False             False      False    False   False   
235792  False   False             False      False    False   False   
235793  False   False             False      False    False   False   
235794  False   False             False      False    False   False   

        NoOfSelfRef  NoOfEmptyRef  NoOfExternalRef  label  
0             False         False            False  False  
1             False         False            False  False  
2             False         False            False  False  
3             False         False            False  False  
4             False         False            False  False  
...             ...           ...              ...    ...  
235790        False         False            False  False  
235791        False         False            False  False  
235792        False         False            False  False  
235793        False         False            False  False  
235794        False         False            False  False  

[235795 rows x 56 columns]>

# 📌 Step 4: Define Features and Target Variable

In [ ]:
X = df.drop(columns=['label'])  # Drop the target column
y = df['label']  # Labels (1 = phishing, 0 = legitimate)


# 📌 Step 5: Train-Test Split (80-20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
df.head()

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,0.522907,...,0,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,0.032650,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.0,0.866667,0.028555,...,0,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.0,1.000000,0.522907,...,1,1,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.0,1.000000,0.079963,...,1,0,1,244,15,34,72,1,85,1


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, classification_report


In [ ]:

# 📌 Step 1: Drop Irrelevant Columns
df = df.drop(columns=['FILENAME', 'URL'], errors='ignore')

# 📌 Step 2: Check and Handle Missing Values
df.dropna(inplace=True)  # Removes all rows with NaN values

# 📌 Step 3: Identify Non-Numeric Columns
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
print("🔎 Non-numeric columns before encoding:", non_numeric_cols)

# 📌 Step 4: Encode Categorical Features
label_encoders = {}
for col in non_numeric_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Convert to string before encoding
    label_encoders[col] = le

# 📌 Step 5: Verify All Columns Are Numeric
print("🔎 Dataset dtypes after encoding:\n", df.dtypes)

# 📌 Step 6: Split Features and Labels
X = df.drop(columns=['label'])  # Features
y = df['label']  # Target

# 📌 Step 7: Ensure `y` Has No NaN Values
assert not y.isnull().any(), "⚠ NaN values found in target variable!"

# 📌 Step 8: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 Step 9: Ensure All Values Are Numeric Before Scaling
assert X_train.select_dtypes(exclude=[np.number]).empty, "⚠ Non-numeric values found after encoding!"

# 📌 Step 10: Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 📌 Step 11: Train a Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_preds = rf_model.predict(X_test_scaled)

# 📌 Step 12: Evaluate Model
from sklearn.metrics import accuracy_score, classification_report
print("🎯 Model Accuracy:", accuracy_score(y_test, rf_preds))
print("\n📌 Classification Report:\n", classification_report(y_test, rf_preds))

🔎 Non-numeric columns before encoding: []
🔎 Dataset dtypes after encoding:
 URLLength                     float64
Domain                          int64
DomainLength                  float64
IsDomainIP                    float64
TLD                             int64
URLSimilarityIndex            float64
CharContinuationRate          float64
TLDLegitimateProb             float64
URLCharProb                   float64
TLDLength                     float64
NoOfSubDomain                 float64
HasObfuscation                float64
NoOfObfuscatedChar            float64
ObfuscationRatio              float64
NoOfLettersInURL              float64
LetterRatioInURL              float64
NoOfDegitsInURL               float64
DegitRatioInURL               float64
NoOfEqualsInURL               float64
NoOfQMarkInURL                float64
NoOfAmpersandInURL            float64
NoOfOtherSpecialCharsInURL    float64
SpacialCharRatioInURL         float64
IsHTTPS                       float64
LineOfCode  

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# 📌 Step 1: Load Dataset
df = pd.read_csv("/content/PhiUSIIL_Phishing_URL_Dataset.csv")  # Replace with your dataset path

# 📌 Step 2: Drop Non-Numeric Features
df = df.drop(columns=['FILENAME', 'URL', 'Domain', 'TLD', 'Title'], errors='ignore')

# 📌 Step 3: Handle Missing Values
df = df.dropna()

# 📌 Step 4: Encode Target Variable
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# 📌 Step 5: Split Features & Target
X = df.drop(columns=['label'])
y = df['label']

# 📌 Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 Step 7: Scale Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 📌 Step 8: Define Autoencoder Model
input_dim = X_train_scaled.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(32, activation='relu')(input_layer)
encoded = Dense(16, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# 📌 Step 9: Train Autoencoder
autoencoder.fit(X_train_scaled, X_train_scaled, epochs=50, batch_size=32, validation_data=(X_test_scaled, X_test_scaled), verbose=1)

# 📌 Step 10: Extract Latent Features
encoder = Model(input_layer, encoded)
X_train_encoded = encoder.predict(X_train_scaled)
X_test_encoded = encoder.predict(X_test_scaled)

# 📌 Step 11: Feature Selection using Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_encoded, y_train)

# Get Feature Importance
feature_importances = rf.feature_importances_
important_indices = np.argsort(feature_importances)[-10:]  # Select Top 10 Features

# Select Important Features
X_train_selected = X_train_encoded[:, important_indices]
X_test_selected = X_test_encoded[:, important_indices]

# 📌 Step 12: Train SVM on Selected Features
svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm.fit(X_train_selected, y_train)

# 📌 Step 13: Predict and Evaluate Model
y_pred = svm.predict(X_test_selected)

# 📌 Step 14: Print Results
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))


Epoch 1/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - loss: 0.7727 - val_loss: 0.7747
Epoch 2/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - loss: 0.6702 - val_loss: 0.7650
Epoch 3/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 0.7286 - val_loss: 0.7611
Epoch 4/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.6981 - val_loss: 0.7591
Epoch 5/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - loss: 0.6656 - val_loss: 0.7581
Epoch 6/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.6975 - val_loss: 0.7577
Epoch 7/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 0.6803 - val_loss: 0.7577
Epoch 8/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 0.7305 - val_loss: 0.7568
Epoch 9/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 0.6558 - val_loss: 0.7563
Epoch 10/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 0.6504 - val_loss: 0.7556
Epoch 11/50
5895/5895 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 0.6997 - val_loss: 0.7556
Epoch 12/50
5895/58